In [50]:
#from fastapi import FastAPI, File, UploadFile, Request
#from pydantic import BaseModel
#from typing import List

#from fastapi.templating import Jinja2Templates
#from fastapi.staticfiles import StaticFiles
#from joblib import load
#from sklearn.preprocessing import OneHotEncoder




from fastapi.encoders import jsonable_encoder

import pickle
import pandas as pd



item = {
  "name": "unknown",
  "year": 2010,
  "selling_price": "unknown",
  "km_driven": 168000,
  "fuel": "Diesel",
  "seller_type": "Individual",
  "transmission": "Manual",
  "owner": "First Owner",
  "mileage": 14.0,
  "engine": 2498,
  "max_power": 112,
  "torque": "unknown",
  "seats": 7
  }


loaded_model = pickle.load(open('model_.pkl','rb'))
onehotencoder = pickle.load(open('encoder_.pkl','rb'))
standardscaler = pickle.load(open('standardscaler_.pkl','rb'))


data = pd.DataFrame(jsonable_encoder(item), index=[0])
data = data.drop(['name', 'selling_price', 'torque'], axis=1)

cat_features_mask = (data.dtypes == "object").values
data_X = data[data.columns[~cat_features_mask]]

data_X_st_np = standardscaler.transform(data_X)

data_X_st = pd.DataFrame(data=data_X_st_np, columns=data_X.columns)
data_X_st_noseat = data_X_st.drop(['seats'], axis = 1)

data['seats'] = data['seats'].astype("object")
cat_features_mask = (data.dtypes == "object").values
data_only_cat = data[data.columns[cat_features_mask]]

data_only_cat_OHE_np = onehotencoder.transform(data_only_cat).toarray()
data_only_cat_OHE = pd.DataFrame(data=data_only_cat_OHE_np)


data2 = pd.concat([data_X_st_noseat, data_only_cat_OHE], axis=1)

prediction = loaded_model.predict(data2)

print('Prediction: ', prediction)

Prediction:  [611351.61093946]


/Users/mike/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
